In [ ]:
# Les oiseaux représentent un atout majeur dans la caractérisation des états écologoiques des 
# écosystèmes. Cependant, il est très difficile de répertorier de manière exhaustive la diversité
# des espèces présentes dans un biome donné. L'objectif de ce travail est donc de construire
# un algorithme deep-learning capable de reconnaître les oiseaux sur la seule base de leur chant. 
# On dispose pour cela d'enregistrements audio au format mp3 correspondant à 264 espèces d'oiseaux. 

# L'objectif dans un premier temps est de pouvoir traiter les fichiers audio afin de les inclure
# par la suite dans l'algorithme de deep-learning.

In [ ]:
# Chargement des librairies classiques et de machine learning dans python
import os 
import numpy as np 
import pandas as pd 
import math
import cv2
import pathlib
import librosa
import librosa.display
import skimage
import skimage.io
from IPython.display import Audio
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from scipy.ndimage.measurements import center_of_mass

# A partir de la librairie Keras, on importe les modules nécessaires au deep-learning
from keras import Sequential
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv2D, MaxPooling2D,  Activation, Flatten, experimental, BatchNormalization, MaxPool2D
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# On ignore les messages d'erreurs des fichiers
warnings.filterwarnings('ignore')

In [ ]:
# PARTIE 1 : Prétraitement des fichiers audios

In [ ]:
# Si besoin, on affiche la liste des chemins d'accès du working directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

In [ ]:
# Tout les fichiers audio sont convertis à la même fréquence 44100 (Hz) pour une 
# randomisation des traitements de fichier audio
config = {
    "sample_rate": 44100 ## 
}

In [ ]:
#Création d'une fonction pour transformer les sons mp3 en spectrogramme de Mel
def spectrogram_image(y, sr,):       
    
    """
    y: audio samples: numpy array (2, n)
    sr: sample rate: number
    """
    
    HOP_SIZE = 1024       
    N_MELS = 128              
    WINDOW_TYPE = 'hann' 
    FEATURE = 'mel'      
    FMIN = 1400 
    
    y_chunks= librosa.effects.split(y) 
    
    mfccs_final = []
    
    for chunk in y_chunks:
        
        mels = librosa.feature.melspectrogram(y=y,sr=sr,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, 
                                        fmax=sr/2) 

        mels = librosa.power_to_db(mels**2,ref=np.max)
        mfccs = librosa.feature.mfcc(S=mels, n_mfcc=40) 

        mfcss_img = np.reshape(mfccs, (*mfccs.shape, 1))
        
        ## resize and rescale image
        resize_and_rescale = tf.keras.Sequential([
            layers.experimental.preprocessing.Resizing(40, 40),
            layers.experimental.preprocessing.Rescaling(1./255)
        ])
        
        mfcss_img = resize_and_rescale(mfcss_img)

        mfcss_image = np.reshape(mfcss_img, (mfcss_img.shape[0], mfcss_img.shape[1]))
        mfccs_final.append(mfcss_image)
    
    return np.array(mfccs_final)

In [ ]:
# Fonction qui retourne le label de l'image
def gen_label_encoder():
    return LabelEncoder()

# Importation du fichier csv, on conserves uniquement les variables qui nous intéresse 
# pour la suite du traitement des données
def save_image(y, out):
    skimage.io.imsave(out, y)

In [ ]:
# Extraire les informations utiles
raw_datasets = pd.read_csv("/kaggle/input/birdsong-recognition/train.csv")

datasets = raw_datasets.loc[:, 
            ['location', 'rating', 'ebird_code', 'duration', 'filename', 'time', 'primary_label', 'sampling_rate',
             'length', 'channels', 'pitch', 'bird_seen', 'background', 'bitrate_of_mp3', 'volume', 'file_type']]

datasets = datasets[datasets.rating >= 4.]

In [ ]:
# Chargement des données
label_encoder = gen_label_encoder()
datasets['duration'] = datasets.duration.astype(float)
datasets['label'] = label_encoder.fit_transform(datasets.ebird_code.to_numpy())

In [ ]:
# Récupération des fichiers audio et passage des fichiers dans les fonctions définies précédemment
# qui traite le son et le transforme en image spectrogramme de Mel.
def data_generator(datasets):
    while True:
        for index, row in datasets.iterrows():
            audio_p = f'/kaggle/input/birdsong-recognition/train_audio/{row.ebird_code}/{row.filename}'
            if os.path.isfile(audio_p):   
                try:
                    audio_numpy, _ = librosa.load(audio_p, mono=True, sr=None)
                    audio_numpy, _ = librosa.effects.trim(audio_numpy, top_db=20)
                    audio_name = row.filename
                    
                    audio_mfccs = spectrogram_image(
                        audio_numpy, 
                        config['sample_rate']
                    )
                    
                    yield ( # Associé chaque image à un label
                        audio_mfccs, 
                        tf.keras.utils.to_categorical(
                            row.label, 
                            num_classes=len(datasets.ebird_code.unique()), 
                        ),
                        row.ebird_code, 
                        row.filename)
                    
                except Exception as e:
                    print(f"ignore error data {audio_name}")
                    raise e
                    pass
        else:
            break

In [ ]:
# Enregistrement des images sous format png dans le chemin output
for mfccs, encoded_y, ebird_code, filename in data_generator(datasets):
           
    HOP_SIZE = 1024       
    N_MELS = 128            
    # On stock les images créées dans un nouveau répertoire
    path = pathlib.Path(f'/kaggle/working/{ebird_code}')
    
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)
          
    index = 0
    [file_path, _] = os.path.splitext(os.path.join(*path.parts, filename))
    for mfcc in mfccs:  
        save_image(mfcc, out=f"{file_path}.{index}.png")
        index += 1

In [ ]:
# Maintenant que les fichiers audio ont été converti en image pnj, il est possible de les traiter
# dans un algorithme de deep-learning.

In [ ]:
# PARTIE 2 : # Traitement des images

In [ ]:
# Définition du chemin d'accès aux données générées précédemment
data_dir = "/kaggle/input/birdsongsrecognitionkevanrastello"
data_path = pathlib.Path(data_dir)

# Obtenir le nombre total d'images 
total_images = len(list(data_path.glob("*/*.png")))

# Afiicher la liste 
total_images

In [ ]:
# Afficher la liste des sous repertoires = la liste des especes
os.listdir("/kaggle/input/birdsongsrecognitionkevanrastello") 
labels = os.listdir("/kaggle/input/birdsongsrecognitionkevanrastello") 

In [ ]:
# Regarder les chemins ou sont stockées les repertoires
list(data_path.glob("*/*.png")) 

In [ ]:
# On plot une image pour visualiser le spectrogramme généré précédemment
image0 = plt.imread('/kaggle/input/birdsongsrecognitionkevanrastello/aldfly/XC135454.0.png')

plt.imshow(image0)

In [ ]:
# Chargement des images dans l'algorithme

# Définition de la taille du batch (nombre d'images utilisées à chaque étape de la descente 
# de gradient pour déterminer la pente à suivre) Il est possible d'augmenter le batch, 32 est 
# une valeur assez faible 
BATCH_SIZE = 64

# Renseigner la taille des images utilisées (hauteur x largeur)
IMG_HEIGHT = 128
IMG_WIDTH = 128
SEED = np.random.randint(100)

# Comme les data sont volumineuses, à chaque etape de l’algorithme de gradient va chercher 
# qlq images et les utilser pour calculer la descente de gradient 
# Ensuite, il les enleve de la mémoire et il recommence

# Création des jeux de données d'apprentissage et de validation (90% de d'apprentissage, 10% de validation)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

In [ ]:
# Vider le cache pour libérer la mémoire vive
cache_train_ds = train_ds.cache().prefetch(tf.data.experimental.AUTOTUNE)
cache_val_ds = val_ds.cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Nombre d'espèces
num_classes = 264

# Création de l'algortihme de deep-learning (réseau de neurone, CNN)
# Répétion de trois motifs, Convolution - Max_pooling - Dropout
# La fonction d'activation choisie est la fonction relu pour sa simplicité d'utilisation 
# L'étape d'applatissement permet de réduire les dimension des données à 1 dimension pour récupérer 
# le label en sortie
# La couche dense permet contrairement à l'apprentissage local de la convolution, 
# de réaliser un apprentissage global sur l'entièreté des images.

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),# Mise à l'échelle entre 0 et 1
  layers.Conv2D(32, 3, activation='relu'), # On utilise 32 filtres de taille 3*3
  layers.MaxPooling2D(),
  layers.Dropout(0.2),# 20% = probabilite d'activation de chaque neurone
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# On définit d'autre propriété du modèle, notamment en utilisant l'optimiseur 'adam'
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
# Définition du nombre d'époques = Nombre de passage de l'ensemble des données d'apprentissage
# dans le réseau de neurones
epochs = 10

# On lance le modèel complet 
history = model.fit(
  cache_train_ds,
  validation_data=val_ds,
  shuffle=True,
  epochs=epochs
)

In [ ]:
# Représenation graphique des métriques calculées par le modèle 
# Fonction de perte et fonction de précision
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

# On trace les fonction de perte et de précision pour les données d'apprentisage et de validation
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# La perte d'apprentissage (training loss) indique la qualité de l'adaptation du modèle aux données d'apprentissage, tandis
#la perte de validataion (validation loss) indique la qualité de l'adaptation du modèle aux données de validation.

#Nous remarquons que la prédiction de notre modèle est d'environ 66%. La perte d'apprentissage est bien inférieur à la perte de
#validatation. En revanche, la perte de validation remonte légèrement à la fin des époques, ce qui semble nous indiquer
#que nous avons un léger problème de sur-apprentissage.

In [ ]:
# On sauvegarde le modèle 
model.save("./model_backup.h5")
np.save("class_indices.npy", np.array(train_ds.class_names))

In [ ]:
# On peut regarder la puissance de prédiction du modèle, c'est à dire la capacité qu'il possède 
# a prédire correctement une image en entrée qu'il n'a jamais vu.
y_input = []
y_output = []

for x, y in cache_val_ds.take(1):
    predicts = model.predict(x)
    for index, y_real in enumerate(y):
        y_pred = predicts[index]
        score = tf.nn.softmax(y_pred)
        print(f'Class: {train_ds.class_names[y_real]} -  Predict as {train_ds.class_names[np.argmax(y_pred)]} with score {np.max(score) * 100}%')
        y_input = np.append(y_input, train_ds.class_names[y_real])
        y_output = np.append(y_output, train_ds.class_names[np.argmax(y_pred)])

In [ ]:
y_input

In [ ]:
y_output

In [ ]:
# Creation de la matrice de confusion pour estimer la capacité prédictive du modèle
# On plot une matrice qui représente la probabilité que notre oiseaux correspond bien à un label
cm = confusion_matrix(y_input, y_output, normalize = 'true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(include_values=True, cmap='cividis')

axes = plt.gca()
axes.xaxis.set_ticklabels(labels, fontsize = 10, verticalalignment = 'center') 
axes.yaxis.set_ticklabels(labels, fontsize = 10, verticalalignment = 'center', rotation = 90)
plt.show()

# Les des oiseaux sont bien prédits quand la diagonale est jaune, tandis que si la couleur est bleu les oiseaux sont mal prédits.

In [ ]:
#Conclusion :
#Il semble d'après la matrice de confusion que les oiseaux possèdant un chant similaire sont davantage mal prédit 
#que les oiseaux étant d'espèce très distincte. Par exemple, le guiraca bleu est très rarement prédit comme la 
#pie américaine. En revanche, il arrive souvent que le guiraca soit confondu avec le merle d'amérique (american robin).